In [1]:
from WindPy import w
import pandas as pd
import os
import copy

In [2]:
w.start()
result=w.isconnected()
print(result)

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2020 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.
True


In [3]:
def other_information_block(code, start, end):
    #issue date
    other_info = pd.DataFrame(w.wsd(code, "stmnote_audit_category,stm_issuingdate", start, end, "zoneType=1;Period=Y;Days=Alldays;Fill=Previous").Data)
    columns = list(other_info.columns)
    columns[0], columns[1] = columns[1], columns[0]
    other_info = other_info[columns]

    #插空行处理，审计意见（境外）+原始报表，后续更新？
    empty_rows = pd.DataFrame([[None] * len(other_info.columns)] * 2, columns=other_info.columns)
    # 将数据框分割成两部分
    other_info1 = other_info.iloc[:1]  # 第1行数据
    other_info2 = other_info.iloc[1:]  # 第2行数据
    # 合并数据框部分和空白行
    other_info = pd.concat([other_info1, empty_rows, other_info2]).reset_index(drop=True)

    #披露日期标准化处理
    # 确保最后一行的指定列是字符串类型
    other_info.iloc[-1, [0, 1]] = other_info.iloc[-1, [0, 1]].astype(str)
    # 使用 pd.to_datetime 转换为日期时间格式，然后只保留日期部分
    other_info.iloc[-1, [0, 1]] = other_info.iloc[-1, [0, 1]].apply(lambda x: pd.to_datetime(x).date() if isinstance(x, str) and ' ' in x else x)
    return other_info

In [4]:
def capital_liability_statement_block(code,start,end,capital_liability_statement):
    #报表细分项目收集，列表形式
    cur_ast_term = ['monetary_cap','tradable_fin_assets','derivative_fin_assets','notes_rcv','acct_rcv','prepay','oth_rcv_tot',
                'inventories','cont_assets','hfs_assets','non_cur_assets_due_within_1y','oth_cur_assets','tot_cur_assets']

    noncur_ast_term = ['debt_invest', 'oth_debt_invest', 'long_term_rec', 'long_term_eqy_invest', 'oth_eqy_instruments_invest',
                    'oth_non_cur_fina_asset', 'invest_real_estate', 'fix_assets', 'const_in_prog', 'productive_bio_assets',
                    'oil_and_natural_gas_assets', 'intang_assets', 'r_and_d_costs', 'goodwill', 'long_term_deferred_exp',
                    'deferred_tax_assets', 'oth_non_cur_assets', 'tot_non_cur_assets', 'tot_assets']
    
    cur_liab_term = ['st_borrow', 'tradable_fin_liab', 'notes_payable', 'acct_payable', 'adv_from_cust', 'cont_liab', 
                       'empl_ben_payable', 'taxes_surcharges_payable', 'oth_payable', 'hfs_liab', 'non_cur_liab_due_within_1y', 
                       'oth_cur_liab', 'tot_cur_liab']
    
    noncur_liab_term = ['lt_borrow', 'bonds_payable', 'lt_payable', 'provisions', 'deferred_inc_non_cur_liab', 'deferred_tax_liab',
                        'oth_non_cur_liab', 'tot_non_cur_liab', 'tot_liab']
    
    equity_term = ['cap_stk', 'other_equity_instruments', 'other_equity_instruments_PRE', 'perpetual_debt', 'cap_rsrv', 
                   'tsy_stk', 'other_compreh_inc_bs', 'special_rsrv', 'surplus_rsrv', 'undistributed_profit', 'minority_int', 
                   'eqy_belongto_parcomsh', 'tot_equity', 'prov_nom_risks', 'tot_liab_shrhldr_eqy']

    # 通过Wind API 提取数据，并储存
    #流动资产
    cap1  = w.wsd(code, 
                  cur_ast_term, 
                  start, 
                  end, 
                  "unit=1;rptType=1;Period=Y;Days=Alldays")
    #非流动资产
    cap2  = w.wsd(code, 
                  noncur_ast_term,
                  start, 
                  end, 
                  "unit=1;rptType=1;Period=Y;Days=Alldays")
    #流动负债
    liab1 = w.wsd(code, 
                  cur_liab_term, 
                  start, 
                  end, 
                  "unit=1;rptType=1;Period=Y;Days=Alldays")
    #非流动负债
    liab2 = w.wsd(code, 
                  noncur_liab_term, 
                  start, 
                  end, 
                  "unit=1;rptType=1;Period=Y;Days=Alldays")
    #所有者权益
    equity = w.wsd(code, 
                  equity_term, 
                  start, 
                  end, 
                  "unit=1;rptType=1;Period=Y;Days=Alldays")

    
    #将储存数据统一为dataframe，并与模板对其时间
    cur_ast = pd.DataFrame(cap1.Data)
    columns = list(cur_ast.columns)
    columns[0], columns[1] = columns[1], columns[0]
    cur_ast = cur_ast[columns]
    
    noncur_ast = pd.DataFrame(cap2.Data)
    columns = list(noncur_ast.columns)
    columns[0], columns[1] = columns[1], columns[0]
    noncur_ast = noncur_ast[columns]
    
    cur_liab = pd.DataFrame(liab1.Data)
    columns = list(cur_liab.columns)
    columns[0], columns[1] = columns[1], columns[0]
    cur_liab = cur_liab[columns]
    
    noncur_liab = pd.DataFrame(liab2.Data)
    columns = list(noncur_liab.columns)
    columns[0], columns[1] = columns[1], columns[0]
    noncur_liab = noncur_liab[columns]
    
    eqt_right = pd.DataFrame(equity.Data)
    columns = list(eqt_right.columns)
    columns[0], columns[1] = columns[1], columns[0]
    eqt_right = eqt_right[columns]

    #在模板特定位置导入相关数据
    capital_liability_statement.iloc[4:4+len(cur_ast_term),1:3] = cur_ast
    capital_liability_statement.iloc[18:18+len(noncur_ast_term),1:3] = noncur_ast
    capital_liability_statement.iloc[38:38+len(cur_liab_term),1:3] = cur_liab
    capital_liability_statement.iloc[52:52+len(noncur_liab_term),1:3] = noncur_liab
    capital_liability_statement.iloc[62:62+len(equity_term),1:3] = eqt_right
    capital_liability_statement.iloc[77:81,1:3] = other_information_block(code,start,end)

    capital_liability_statement.columns = [code, end, start]

    return capital_liability_statement

In [5]:
def adj_capital_liability_statement_block(code,start,end,adjcapital_liability_statement):
    #报表细分项目收集，列表形式
    cur_ast_term = ['monetary_cap','tradable_fin_assets','derivative_fin_assets','notes_rcv','acct_rcv','prepay','oth_rcv_tot',
                'inventories','cont_assets','hfs_assets','non_cur_assets_due_within_1y','oth_cur_assets','tot_cur_assets']

    noncur_ast_term = ['debt_invest', 'oth_debt_invest', 'long_term_rec', 'long_term_eqy_invest', 'oth_eqy_instruments_invest',
                    'oth_non_cur_fina_asset', 'invest_real_estate', 'fix_assets', 'const_in_prog', 'productive_bio_assets',
                    'oil_and_natural_gas_assets', 'intang_assets', 'r_and_d_costs', 'goodwill', 'long_term_deferred_exp',
                    'deferred_tax_assets', 'oth_non_cur_assets', 'tot_non_cur_assets', 'tot_assets']
    
    cur_liab_term = ['st_borrow', 'tradable_fin_liab', 'notes_payable', 'acct_payable', 'adv_from_cust', 'cont_liab', 
                       'empl_ben_payable', 'taxes_surcharges_payable', 'oth_payable', 'hfs_liab', 'non_cur_liab_due_within_1y', 
                       'oth_cur_liab', 'tot_cur_liab']
    
    noncur_liab_term = ['lt_borrow', 'bonds_payable', 'lt_payable', 'provisions', 'deferred_inc_non_cur_liab', 'deferred_tax_liab',
                        'oth_non_cur_liab', 'tot_non_cur_liab', 'tot_liab']
    
    equity_term = ['cap_stk', 'other_equity_instruments', 'other_equity_instruments_PRE', 'perpetual_debt', 'cap_rsrv', 
                   'tsy_stk', 'other_compreh_inc_bs', 'special_rsrv', 'surplus_rsrv', 'undistributed_profit', 'minority_int', 
                   'eqy_belongto_parcomsh', 'tot_equity', 'prov_nom_risks', 'tot_liab_shrhldr_eqy']

 
    #合并报表（调整）数据摘取，rptType=3
    #流动资产
    adjcap1  = w.wsd(code, 
                  cur_ast_term, 
                  start, 
                  end, 
                  "unit=1;rptType=3;Period=Y;Days=Alldays")
    #非流动资产
    adjcap2  = w.wsd(code, 
                  noncur_ast_term,
                  start, 
                  end, 
                  "unit=1;rptType=3;Period=Y;Days=Alldays")
    #流动负债
    adjliab1 = w.wsd(code, 
                  cur_liab_term, 
                  start, 
                  end, 
                  "unit=1;rptType=3;Period=Y;Days=Alldays")
    #非流动负债
    adjliab2 = w.wsd(code, 
                  noncur_liab_term, 
                  start, 
                  end, 
                  "unit=1;rptType=3;Period=Y;Days=Alldays")
    #所有者权益
    adjequity = w.wsd(code, 
                  equity_term, 
                  start, 
                  end, 
                  "unit=1;rptType=3;Period=Y;Days=Alldays")

    #将储存数据统一为dataframe，并与模板对其时间
    adjcur_ast = pd.DataFrame(adjcap1.Data)
    columns = list(adjcur_ast.columns)
    columns[0], columns[1] = columns[1], columns[0]
    adjcur_ast = adjcur_ast[columns]
    
    adjnoncur_ast = pd.DataFrame(adjcap2.Data)
    columns = list(adjnoncur_ast.columns)
    columns[0], columns[1] = columns[1], columns[0]
    adjnoncur_ast = adjnoncur_ast[columns]
    
    adjcur_liab = pd.DataFrame(adjliab1.Data)
    columns = list(adjcur_liab.columns)
    columns[0], columns[1] = columns[1], columns[0]
    adjcur_liab = adjcur_liab[columns]
    
    adjnoncur_liab = pd.DataFrame(adjliab2.Data)
    columns = list(adjnoncur_liab.columns)
    columns[0], columns[1] = columns[1], columns[0]
    adjnoncur_liab = adjnoncur_liab[columns]
    
    adjeqt_right = pd.DataFrame(adjequity.Data)
    columns = list(adjeqt_right.columns)
    columns[0], columns[1] = columns[1], columns[0]
    adjeqt_right = adjeqt_right[columns]

    #在模板特定位置导入相关数据
    adjcapital_liability_statement.iloc[4:4+len(cur_ast_term),1:3] = adjcur_ast
    adjcapital_liability_statement.iloc[18:18+len(noncur_ast_term),1:3] = adjnoncur_ast
    adjcapital_liability_statement.iloc[38:38+len(cur_liab_term),1:3] = adjcur_liab
    adjcapital_liability_statement.iloc[52:52+len(noncur_liab_term),1:3] = adjnoncur_liab
    adjcapital_liability_statement.iloc[62:62+len(equity_term),1:3] = adjeqt_right
    adjcapital_liability_statement.iloc[77:81,1:3] = other_information_block(code,start,end)

    adjcapital_liability_statement.columns = [code, end, start]

    
    return adjcapital_liability_statement

In [6]:
def cashflow_statement_block(code,start,end,cashflow_statement):
    #报表细分项目收集，列表形式
    op_term = ['cash_recp_sg_and_rs', 'recp_tax_rends', 'other_cash_recp_ral_oper_act', 'stot_cash_inflows_oper_act', 
           'cash_pay_goods_purch_serv_rec', 'cash_pay_beh_empl', 'pay_all_typ_tax', 'other_cash_pay_ral_oper_act', 
           'stot_cash_outflows_oper_act', 'net_cash_flows_oper_act']

    inv_term = ['cash_recp_disp_withdrwl_invest', 'cash_recp_return_invest', 'net_cash_recp_disp_fiolta', 
                'net_cash_recp_disp_sobu', 'other_cash_recp_ral_inv_act', 'stot_cash_inflows_inv_act', 
                'cash_pay_acq_const_fiolta', 'cash_paid_invest', 'net_cash_pay_aquis_sobu', 'other_cash_pay_ral_inv_act', 
                'stot_cash_outflows_inv_act', 'net_cash_flows_inv_act']
    
    fin_term = ['cash_recp_cap_contrib', 'cash_recp_borrow', 'other_cash_recp_ral_fnc_act', 'stot_cash_inflows_fnc_act', 
                'cash_prepay_amt_borr', 'cash_pay_dist_dpcp_int_exp', 'other_cash_pay_ral_fnc_act', 'stot_cash_outflows_fnc_act', 
                'net_cash_flows_fnc_act', 'eff_fx_flu_cash', 'net_incr_cash_cash_equ_dm', 'cash_cash_equ_beg_period', 
                'cash_cash_equ_end_period', 'net_incr_cash_cash_equ_gap']
    
    
    sup1_term = ['net_profit_cs', 'prov_depr_assets', 'depr_fa_coga_dpba', 'amort_intang_assets', 'amort_lt_deferred_exp', 
                 'decr_deferred_exp', 'incr_acc_exp', 'loss_disp_fiolta', 'loss_scr_fa', 'loss_fv_chg', 'fin_exp_cs', 
                 'invest_loss', 'decr_deferred_inc_tax_assets', 'incr_deferred_inc_tax_liab', 'decr_inventories',  
                 'decr_oper_payable', 'incr_oper_payable', 'unconfirmed_invest_loss_cs', 'others']
    
    sup2_term = ['conv_debt_into_cap', 'conv_corp_bonds_due_within_1y', 'fa_fnc_leases']
    
    sup3_term = ['end_bal_cash', 'beg_bal_cash', 'end_bal_cash_equ', 'beg_bal_cash_equ', 'net_incr_cash_cash_equ_im']


    # 通过Wind API 提取数据，并储存

    #经营活动
    op  = w.wsd(code, 
                  op_term, 
                  start, 
                  end, 
                  "unit=1;rptType=1;Period=Y;Days=Alldays")
    
    #投资活动
    inv  = w.wsd(code, 
                  inv_term,
                  start, 
                  end, 
                  "unit=1;rptType=1;Period=Y;Days=Alldays")
    #筹资活动
    fin = w.wsd(code, 
                  fin_term, 
                  start, 
                  end, 
                  "unit=1;rptType=1;Period=Y;Days=Alldays")
    #补充1-3
    sup1 = w.wsd(code, 
                  sup1_term, 
                  start, 
                  end, 
                  "unit=1;rptType=1;Period=Y;Days=Alldays")
    sup2 = w.wsd(code, 
                  sup2_term, 
                  start, 
                  end, 
                  "unit=1;rptType=1;Period=Y;Days=Alldays")
    sup3 = w.wsd(code, 
                  sup3_term, 
                  start, 
                  end, 
                  "unit=1;rptType=1;Period=Y;Days=Alldays")

    #将储存数据统一为dataframe，并与模板对其时间
    operate = pd.DataFrame(op.Data)
    columns = list(operate.columns)
    columns[0], columns[1] = columns[1], columns[0]
    operate = operate[columns]
    
    invest = pd.DataFrame(inv.Data)
    columns = list(invest.columns)
    columns[0], columns[1] = columns[1], columns[0]
    invest = invest[columns]
    
    finance = pd.DataFrame(fin.Data)
    columns = list(finance.columns)
    columns[0], columns[1] = columns[1], columns[0]
    finance = finance[columns]
    
    supplement1 = pd.DataFrame(sup1.Data)
    columns = list(supplement1.columns)
    columns[0], columns[1] = columns[1], columns[0]
    supplement1 = supplement1[columns]
    
    supplement2 = pd.DataFrame(sup2.Data)
    columns = list(supplement2.columns)
    columns[0], columns[1] = columns[1], columns[0]
    supplement1 = supplement1[columns]
    
    supplement3 = pd.DataFrame(sup3.Data)
    columns = list(supplement3.columns)
    columns[0], columns[1] = columns[1], columns[0]
    supplement3 = supplement3[columns]

    #在模板特定位置导入相关数据
    cashflow_statement.iloc[4:4+len(op_term),1:3] = operate
    cashflow_statement.iloc[15:15+len(inv_term),1:3] = invest
    cashflow_statement.iloc[28:28+len(fin_term),1:3] = finance
    cashflow_statement.iloc[44:44+len(sup1_term),1:3] = supplement1
    cashflow_statement.iloc[64:64+len(sup2_term),1:3] = supplement2
    cashflow_statement.iloc[68:68+len(sup3_term),1:3] = supplement3
    cashflow_statement.iloc[73:77,1:3] = other_information_block(code,start,end)

    cashflow_statement.columns = [code, end, start]
    
    return cashflow_statement

In [12]:
def adj_cashflow_statement_block(code,start,end,adjcashflow_statement):
    #报表细分项目收集，列表形式
    op_term = ['cash_recp_sg_and_rs', 'recp_tax_rends', 'other_cash_recp_ral_oper_act', 'stot_cash_inflows_oper_act', 
           'cash_pay_goods_purch_serv_rec', 'cash_pay_beh_empl', 'pay_all_typ_tax', 'other_cash_pay_ral_oper_act', 
           'stot_cash_outflows_oper_act', 'net_cash_flows_oper_act']

    inv_term = ['cash_recp_disp_withdrwl_invest', 'cash_recp_return_invest', 'net_cash_recp_disp_fiolta', 
                'net_cash_recp_disp_sobu', 'other_cash_recp_ral_inv_act', 'stot_cash_inflows_inv_act', 
                'cash_pay_acq_const_fiolta', 'cash_paid_invest', 'net_cash_pay_aquis_sobu', 'other_cash_pay_ral_inv_act', 
                'stot_cash_outflows_inv_act', 'net_cash_flows_inv_act']
    
    fin_term = ['cash_recp_cap_contrib', 'cash_recp_borrow', 'other_cash_recp_ral_fnc_act', 'stot_cash_inflows_fnc_act', 
                'cash_prepay_amt_borr', 'cash_pay_dist_dpcp_int_exp', 'other_cash_pay_ral_fnc_act', 'stot_cash_outflows_fnc_act', 
                'net_cash_flows_fnc_act', 'eff_fx_flu_cash', 'net_incr_cash_cash_equ_dm', 'cash_cash_equ_beg_period', 
                'cash_cash_equ_end_period', 'net_incr_cash_cash_equ_gap']
    
    
    sup1_term = ['net_profit_cs', 'prov_depr_assets', 'depr_fa_coga_dpba', 'amort_intang_assets', 'amort_lt_deferred_exp', 
                 'decr_deferred_exp', 'incr_acc_exp', 'loss_disp_fiolta', 'loss_scr_fa', 'loss_fv_chg', 'fin_exp_cs', 
                 'invest_loss', 'decr_deferred_inc_tax_assets', 'incr_deferred_inc_tax_liab', 'decr_inventories',  
                 'decr_oper_payable', 'incr_oper_payable', 'unconfirmed_invest_loss_cs', 'others']
    
    sup2_term = ['conv_debt_into_cap', 'conv_corp_bonds_due_within_1y', 'fa_fnc_leases']
    
    sup3_term = ['end_bal_cash', 'beg_bal_cash', 'end_bal_cash_equ', 'beg_bal_cash_equ', 'net_incr_cash_cash_equ_im']


    # 通过Wind API 提取数据，并储存

    #经营活动
    adjop  = w.wsd(code, 
                  op_term, 
                  start, 
                  end, 
                  "unit=1;rptType=3;Period=Y;Days=Alldays")
    
    #投资活动
    adjinv  = w.wsd(code, 
                  inv_term,
                  start, 
                  end, 
                  "unit=1;rptType=3;Period=Y;Days=Alldays")
    #筹资活动
    adjfin = w.wsd(code, 
                  fin_term, 
                  start, 
                  end, 
                  "unit=1;rptType=3;Period=Y;Days=Alldays")
    #补充1-3
    adjsup1 = w.wsd(code, 
                  sup1_term, 
                  start, 
                  end, 
                  "unit=1;rptType=3;Period=Y;Days=Alldays")
    adjsup2 = w.wsd(code, 
                  sup2_term, 
                  start, 
                  end, 
                  "unit=1;rptType=3;Period=Y;Days=Alldays")
    adjsup3 = w.wsd(code, 
                  sup3_term, 
                  start, 
                  end, 
                  "unit=1;rptType=3;Period=Y;Days=Alldays")

    #将储存数据统一为dataframe，并与模板对其时间
    adjoperate = pd.DataFrame(adjop.Data)
    columns = list(adjoperate.columns)
    columns[0], columns[1] = columns[1], columns[0]
    adjoperate = adjoperate[columns]
    
    adjinvest = pd.DataFrame(adjinv.Data)
    columns = list(adjinvest.columns)
    columns[0], columns[1] = columns[1], columns[0]
    adjinvest = adjinvest[columns]
    
    adjfinance = pd.DataFrame(adjfin.Data)
    columns = list(adjfinance.columns)
    columns[0], columns[1] = columns[1], columns[0]
    adjfinance = adjfinance[columns]
    
    adjsupplement1 = pd.DataFrame(adjsup1.Data)
    columns = list(adjsupplement1.columns)
    columns[0], columns[1] = columns[1], columns[0]
    adjsupplement1 = adjsupplement1[columns]
    
    adjsupplement2 = pd.DataFrame(adjsup2.Data)
    columns = list(adjsupplement2.columns)
    columns[0], columns[1] = columns[1], columns[0]
    adjsupplement1 = adjsupplement1[columns]
    
    adjsupplement3 = pd.DataFrame(adjsup3.Data)
    columns = list(adjsupplement3.columns)
    columns[0], columns[1] = columns[1], columns[0]
    adjsupplement3 = adjsupplement3[columns]

    #在模板特定位置导入相关数据
    adjcashflow_statement.iloc[4:4+len(op_term),1:3] = adjoperate
    adjcashflow_statement.iloc[15:15+len(inv_term),1:3] = adjinvest
    adjcashflow_statement.iloc[28:28+len(fin_term),1:3] = adjfinance
    adjcashflow_statement.iloc[44:44+len(sup1_term),1:3] = adjsupplement1
    adjcashflow_statement.iloc[64:64+len(sup2_term),1:3] = adjsupplement2
    adjcashflow_statement.iloc[68:68+len(sup3_term),1:3] = adjsupplement3
    adjcashflow_statement.iloc[73:77,1:3] = other_information_block(code,start,end)

    adjcashflow_statement.columns = [code, end, start]
    
    return adjcashflow_statement

In [17]:
def income_statement_block(code,start,end,income_statement):
    #报表细分项目收集，列表形式
    op_inc_term = ['tot_oper_rev', 'oper_rev', 'other_oper_inc', 'int_inc', 'handling_chrg_comm_inc', 'insur_prem_unearned']
    
    
    op_cost_term = ['operating_cost2', 'oper_cost', 'taxes_surcharges_ops', 'selling_dist_exp', 'gerl_admin_exp', 'rd_exp',
                    'fin_exp_is', 'impair_loss_assets', 'credit_impair_loss', 'other_oper_exp', 'int_exp', 'handling_chrg_comm_exp',
                    'prepay_surr', 'net_claim_exp', 'net_insur_cont_rsrv', 'dvd_exp_insured', 'reinsurance_exp', 'other_grants_inc', 
                    'net_invest_inc', 'inc_invest_assoc_jv_entp', 'net_exposure_hedge_ben', 'net_gain_chg_fv', 'gain_asset_dispositions', 
                    'net_gain_fx_trans', 'opprofit_gap']
    
    
    op_rev_term = ['opprofit', 'non_oper_rev', 'non_oper_exp', 'net_loss_disp_noncur_asset', 'profit_gap', 'tot_profit',
                   'tax', 'unconfirmed_invest_loss_is', 'net_profit_is_gap']
    
    
    net_rev_term = ['net_profit_is','minority_int_inc','np_belongto_parcomsh','other_compreh_inc','tot_compreh_inc',
                    'tot_compreh_inc_parent_comp','tot_compreh_inc_min_shrhldr','eps_basic_is','eps_diluted_is']


    # 通过Wind API 提取数据，并储存

    #营业收入
    op_inc  = w.wsd(code, 
                  op_inc_term, 
                  start, 
                  end, 
                  "unit=1;rptType=1;Period=Y;Days=Alldays")
    
    #营业成本
    op_cost  = w.wsd(code, 
                  op_cost_term,
                  start, 
                  end, 
                  "unit=1;rptType=1;Period=Y;Days=Alldays")
    #营业利润
    op_rev = w.wsd(code, 
                  op_rev_term, 
                  start, 
                  end, 
                  "unit=1;rptType=1;Period=Y;Days=Alldays")
    #净利润
    net_rev = w.wsd(code, 
                  net_rev_term, 
                  start, 
                  end, 
                  "unit=1;rptType=1;Period=Y;Days=Alldays")

    #将储存数据统一为dataframe，并与模板对其时间
    income = pd.DataFrame(op_inc.Data)
    columns = list(income.columns)
    columns[0], columns[1] = columns[1], columns[0]
    income = income[columns]
    
    cost = pd.DataFrame(op_cost.Data)
    columns = list(cost.columns)
    columns[0], columns[1] = columns[1], columns[0]
    cost = cost[columns]
    
    oprevenue = pd.DataFrame(op_rev.Data)
    columns = list(oprevenue.columns)
    columns[0], columns[1] = columns[1], columns[0]
    oprevenue = oprevenue[columns]
    
    netrevenue = pd.DataFrame(net_rev.Data)
    columns = list(netrevenue.columns)
    columns[0], columns[1] = columns[1], columns[0]
    netrevenue = netrevenue[columns]

    #在模板特定位置导入相关数据
    income_statement.iloc[3:3+len(op_inc_term),1:3] = income
    income_statement.iloc[9:9+len(op_cost_term),1:3] = cost
    income_statement.iloc[34:34+len(op_rev_term),1:3] = oprevenue
    income_statement.iloc[43:43+len(net_rev_term),1:3] = netrevenue
    income_statement.iloc[53:57,1:3] = other_information_block(code,start,end)

    income_statement.columns = [code, end, start]
    
    return income_statement

In [13]:
def adj_income_statement_block(code,start,end,adjincome_statement):
    #报表细分项目收集，列表形式
    op_inc_term = ['tot_oper_rev', 'oper_rev', 'other_oper_inc', 'int_inc', 'handling_chrg_comm_inc', 'insur_prem_unearned']
    
    
    op_cost_term = ['operating_cost2', 'oper_cost', 'taxes_surcharges_ops', 'selling_dist_exp', 'gerl_admin_exp', 'rd_exp',
                    'fin_exp_is', 'impair_loss_assets', 'credit_impair_loss', 'other_oper_exp', 'int_exp', 'handling_chrg_comm_exp',
                    'prepay_surr', 'net_claim_exp', 'net_insur_cont_rsrv', 'dvd_exp_insured', 'reinsurance_exp', 'other_grants_inc', 
                    'net_invest_inc', 'inc_invest_assoc_jv_entp', 'net_exposure_hedge_ben', 'net_gain_chg_fv', 'gain_asset_dispositions', 
                    'net_gain_fx_trans', 'opprofit_gap']
    
    
    op_rev_term = ['opprofit', 'non_oper_rev', 'non_oper_exp', 'net_loss_disp_noncur_asset', 'profit_gap', 'tot_profit',
                   'tax', 'unconfirmed_invest_loss_is', 'net_profit_is_gap']
    
    
    net_rev_term = ['net_profit_is','minority_int_inc','np_belongto_parcomsh','other_compreh_inc','tot_compreh_inc',
                    'tot_compreh_inc_parent_comp','tot_compreh_inc_min_shrhldr','eps_basic_is','eps_diluted_is']


    # 通过Wind API 提取数据，并储存

    #营业收入
    adjop_inc  = w.wsd(code, 
                  op_inc_term, 
                  start, 
                  end, 
                  "unit=1;rptType=3;Period=Y;Days=Alldays")
    
    #营业成本
    adjop_cost  = w.wsd(code, 
                  op_cost_term,
                  start, 
                  end, 
                  "unit=1;rptType=3;Period=Y;Days=Alldays")
    #营业利润
    adjop_rev = w.wsd(code, 
                  op_rev_term, 
                  start, 
                  end, 
                  "unit=1;rptType=3;Period=Y;Days=Alldays")
    #净利润
    adjnet_rev = w.wsd(code, 
                  net_rev_term, 
                  start, 
                  end, 
                  "unit=1;rptType=3;Period=Y;Days=Alldays")

    #将储存数据统一为dataframe，并与模板对其时间
    adjincome = pd.DataFrame(adjop_inc.Data)
    columns = list(adjincome.columns)
    columns[0], columns[1] = columns[1], columns[0]
    adjincome = adjincome[columns]
    
    adjcost = pd.DataFrame(adjop_cost.Data)
    columns = list(adjcost.columns)
    columns[0], columns[1] = columns[1], columns[0]
    adjcost = adjcost[columns]
    
    adjoprevenue = pd.DataFrame(adjop_rev.Data)
    columns = list(adjoprevenue.columns)
    columns[0], columns[1] = columns[1], columns[0]
    adjoprevenue = adjoprevenue[columns]
    
    adjnetrevenue = pd.DataFrame(adjnet_rev.Data)
    columns = list(adjnetrevenue.columns)
    columns[0], columns[1] = columns[1], columns[0]
    adjnetrevenue = adjnetrevenue[columns]

    #在模板特定位置导入相关数据
    adjincome_statement.iloc[3:3+len(op_inc_term),1:3] = adjincome
    adjincome_statement.iloc[9:9+len(op_cost_term),1:3] = adjcost
    adjincome_statement.iloc[34:34+len(op_rev_term),1:3] = adjoprevenue
    adjincome_statement.iloc[43:43+len(net_rev_term),1:3] = adjnetrevenue
    adjincome_statement.iloc[53:57,1:3] = other_information_block(code,start,end)

    adjincome_statement.columns = [code, end, start]
    
    return adjincome_statement

In [15]:
def financial_statements_autoextract(code,start,end,template_address, export_address):
    capital_liability_statement = pd.read_excel(template_address, sheet_name=0)
    capital_liability_statement_ori = copy.deepcopy(capital_liability_statement)
    capital_liability_statement_adj = copy.deepcopy(capital_liability_statement)
    
    cashflow_statement = pd.read_excel(template_address, sheet_name=1)
    cashflow_statement_ori = copy.deepcopy(cashflow_statement)
    cashflow_statement_adj = copy.deepcopy(cashflow_statement)
    
    income_statement = pd.read_excel(template_address, sheet_name=2)
    income_statement_ori = copy.deepcopy(income_statement)
    income_statement_adj = copy.deepcopy(income_statement)

    CLS = capital_liability_statement_block(code,start,end,capital_liability_statement_ori)
    adjCLS = adj_capital_liability_statement_block(code,start,end,capital_liability_statement_adj)
    final_CLS = adjCLS.combine_first(CLS)

    CFS = cashflow_statement_block(code,start,end,cashflow_statement_ori)
    adjCFS = adj_cashflow_statement_block(code,start,end,cashflow_statement_adj)
    final_CFS = adjCFS.combine_first(CFS)

    INCS = income_statement_block(code,start,end,income_statement_ori)
    adjINCS = adj_income_statement_block(code,start,end,income_statement_adj)
    final_INCS = adjINCS.combine_first(INCS)

    export_address = f"{export_address}{'financial_statements_'}{code}{'.xlsx'}"
    
    with pd.ExcelWriter(export_address, engine='openpyxl') as writer:
    # 将每个数据框写入不同的工作表
        final_CLS.to_excel(writer, sheet_name='大陆资产负债表', index=False)
        final_CFS.to_excel(writer, sheet_name='大陆现金流量表', index=False)
        final_INCS.to_excel(writer, sheet_name='大陆利润表', index=False)

In [18]:
financial_statements_autoextract("102482383.IB","2022-12-31", "2023-12-31",
                                 'P:/Desktop/Intern/SWHY Fintech/一般财报模板.xlsx',
                                 'P:/Desktop/Intern/SWHY Fintech/')

In [68]:
w.wsd("102482383.IB", "tot_liab_shrhldr_eqy", "2022-12-31", "2023-12-31", "unit=1;rptType=1;Period=Y;Days=Alldays").Data

[[2721076400.0, 89483280903.66]]

In [30]:
w.wsd("102482383.IB", "tot_liab_shrhldr_eqy", "2022-12-31", "2023-12-31", "unit=1;rptType=3;Period=Y;Days=Alldays").Data

[[8388293770.49, nan]]